In [15]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform

In [16]:
def get_reform(amount, double_joint=False):
    joint_amount = amount * 2 if double_joint else amount
    return Reform.from_dict({
    "gov.contrib.crfb.ss_credit.in_effect": {
        "2026-01-01.2100-12-31": True
    },
    "gov.contrib.crfb.ss_credit.amount.SINGLE": {
        "2026-01-01.2100-12-31": amount
    },  
    "gov.contrib.crfb.ss_credit.amount.JOINT": {
        "2026-01-01.2100-12-31": joint_amount
    },  
    "gov.contrib.crfb.ss_credit.amount.SEPARATE": {
        "2026-01-01.2100-12-31": amount
    },  
    "gov.contrib.crfb.ss_credit.amount.HEAD_OF_HOUSEHOLD": {
        "2026-01-01.2100-12-31": amount
    },  
    "gov.contrib.crfb.ss_credit.amount.SURVIVING_SPOUSE": {
        "2026-01-01.2100-12-31": amount
    },  
    }, country_id="us")


In [17]:
AMOUNTS = [300, 600, 900, 1_200]

In [18]:
baseline = Microsimulation()
baseline_income = baseline.calculate("household_net_income", period=2026)


In [19]:
results = []

In [20]:
for amount in AMOUNTS:
    for double_joint, case_name in [(False, 'Flat amount by filing status'), (True, 'Joint amount doubled')]:
        reform = get_reform(amount, double_joint)
        reformed = Microsimulation(reform=reform)
        reformed_income = reformed.calculate("household_net_income", period=2026)
        difference_income = reformed_income - baseline_income
        dif = difference_income.sum() / 1e9
        results.append([amount, case_name, f'${round(dif, 2)}'])

In [21]:
import pandas as pd

In [22]:
df = pd.DataFrame(results, columns=['Credit Amount', 'Case', 'Cost (Billions)'])
print(df)


   Credit Amount                          Case Cost (Billions)
0            300  Flat amount by filing status         $-13.08
1            300          Joint amount doubled          $-8.29
2            600  Flat amount by filing status          $-6.39
3            600          Joint amount doubled           $2.21
4            900  Flat amount by filing status          $-0.19
5            900          Joint amount doubled          $11.03
6           1200  Flat amount by filing status           $5.53
7           1200          Joint amount doubled          $18.26
